# Using hcubature on uniform mesh on spherical shell 

Goal of this notebook is to perform integration in spherical coordinates. We perform computations in two steps: 

1. explicit computation: given the r and theta componenten of the magnetization vector, compute (retrieve) the phi component of the magnetic vector potential and verify that the r and theta componenten of the  magnetic vector potential are zero; 
2. implicit computation: compute the MoM matrix as a Jacobian of the mapping from M to A; 

## Import Packages

In [50]:
using ForwardDiff
using QuadGK
using HCubature 
using StaticArrays 
using LinearAlgebra
using BenchmarkTools
using Plots

In [4]:
# a point in 3D is a tuple of 3 coordinates 
# we here introduce static vectors that appear to be vital to reduce the number of allocations 
const Point3D = SVector{3,Float64};

## Section 1: Definition Vector Potential Density

<b>Introduction</b> 
Here we define the kernel of integration or the vector potential density $\text{vp-dens}$ in spherical coordinates $(r,\theta,\phi)$. As in the cartesian case, the function $\text{vp-dens}$ takes the following three arguments as input:
1. the position vector of the observer (or destination) $\vec{r}(r,\theta, \phi)$;
2. the position vector of the source $\vec{r}_p(r_p,\theta_p,\phi_p)$;
3. the magnetization vector $\vec{M}$ evaluated in the coordinates of the source. 

The function $\text{vp-dens}$ is given by 

$$ 
\begin{eqnarray}
\text{vp-dens}[\vec{r}, \vec{r}_p, \vec{M}] & = & 
   \text{vp-dens}[\vec{r}(r,\theta,\phi), 
                  \vec{r}_p(r_p,\theta_p,\phi_p), \vec{M}(r_p,\theta_p,\phi_p)] \\ 
   & = & \frac{\vec{M}(r_p,\theta_p,\phi_p) \times (\vec{r} - \vec{r}_p)}
        {\| \vec{r}(r,\theta,\phi) - \vec{r}_p(r_p,\theta_p,\phi_p) \|}
\end{eqnarray}
$$

The magnetic vector potential $\vec{A}(r,\theta,\phi)$ is then obtained by integrating its density over the source domain, i.e., by 

$$ 
   \vec{A}(r,\theta,\phi) = 
   \int_{R_a}^{R_b} \int_0^{2\pi} \int_{-\pi/2}^{\pi/2}
   \text{vp-dens}[\vec{r}, \vec{r}_p, \vec{M}] \, 
   r^2 \sin(\theta) \, dr \, d\theta \, d\phi 
$$

Note that integration over the source domain returns a vector with 3 components (the function hcubature.jl is indeed able to integrate vector-valued functions). The resulting vector can be expressed in either carthesian or spherical coordinates.   

<b>Analytical reference solution</b>
In the test case for a spherical shell, the magnetization vector $\vec{M}(r_p,\theta_p,\phi_p)$ only has a $r$-component and a $\theta$ component. 
$$
\vec{M}(r_p,\theta_p,\phi_p) = [M_r, M_{\theta}, 0] 
$$
(place here results obtained from the curl of the vector potential). 

The  magnetic vector potential only has a $\phi$-component is given by 

$$
\vec{A}(r,\theta,\phi) = [0, 0, A_(\phi)]  
$$
(place here solution provided by Eugene)

<b>Pre-processing stage in function computing the vector-potential density</b>
The position vectors $\vec{r}$ and $\vec{r}_p$ can be expressed in either cartesian or spherical unit vector. For $\vec{r}$ for example, the expansion in spherical basis vectors is $\vec{r}(r,\theta,\phi) = r \, \vec{e}_r(r,\theta,\phi)$. Similarly for $\vec{r}_p(r_p,\theta_p,\phi_p) = r_p \, \vec{e}_r(r_p,\theta_p,\phi_p)$. Expression the vector of the difference $\vec{r}(r,\theta,\phi) - \vec{r}_p(r_p,\theta_p,\phi_p)$ in spherical basis vectors would require tranporting $\vec{r}_p$ to the same position as $\vec{r}_p$ (or the vice-versa). To avoid this nuisance, we express $\vec{r}$ and $\vec{r}_p$ in carthesian basis vectors. For $\vec{r}$ we have that 
$$ \vec{r}(r,\theta,\phi) = r \, \vec{e}_r(r,\theta,\phi) = 
   r \cos(\theta) \sin(\phi) \, \vec{\iota} + r \sin(\theta) \sin(\phi) \, \vec{j} 
   + r \cos(\phi) \, \vec{k}$$

This can be asccomplished by a function <b>PositiontoCartesian</b>. 

Need to transform vector components back to spherical basis vectors. 


Expressions for [spherical coordinates](https://dynref.engr.illinois.edu/rvs.html). 

In [10]:
# define vector potential density 
function vp_density(r, rprime, M)
    num = cross(M, (r - rprime))
    denom = norm(r - rprime)^3
    result = num/denom
    return result   
end

vp_density (generic function with 1 method)

In [13]:
M = [0.,0.,1.]
r = [1.,0.,0.]
rp = [0.,1.,0.]
vp_density(r,rp,M)

3-element Vector{Float64}:
  0.3535533905932737
  0.3535533905932737
 -0.0

In [16]:
M = [0.,0.,1.]
r = [1.,0.,0.]
t = 0.5 
hcubature(rp -> vp_density(r,rp,M), (0.0, 0.0, -t), (1.0, 1.0, t))[1]

3-element Vector{Float64}:
 1.551694097302116
 1.5516940972927296
 0.0

## Section 2: Perform Integration over Mesh 

In [17]:
# loop over midpoints of elements 
N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
for xi in xmesh[1:end-1] 
    for yj in ymesh[1:end-1] 
      mid = Point3D(xi+h/2,yj+h/2,0)
      # display(mid)
   end 
end 

In [52]:
# test integration assuming integrand for which answer can be checked analytically 
function settoone(rp)
    return 1.0
end 

N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = zeros(N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      mid = Point3D(xi+h/2,yj+h/2,0)
      result[i,j] = hcubature(rp -> settoone(rp), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end

In [49]:
result

5×5 Matrix{Float64}:
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04
 0.04  0.04  0.04  0.04  0.04

In [53]:
# assume r to coincide with midpoint of the element
N = 100; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = fill(zeros(3),N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      mid = Point3D(xi+h/2,yj+h/2,0)
      result[i,j] = hcubature(rp -> vp_density(mid,rp,M), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end

In [41]:
result

5×5 Matrix{Vector{Float64}}:
 [-8.05737e-17, 1.0516e-12, 0.0]   …  [4.33192e-18, 1.06581e-12, 0.0]
 [-7.91297e-17, 5.68434e-14, 0.0]     [-1.44397e-18, 7.10543e-14, 0.0]
 [-8.34616e-17, -9.9476e-14, 0.0]     [-2.88794e-18, -1.13687e-13, 0.0]
 [-4.33192e-18, -9.9476e-14, 0.0]     [0.0, -1.13687e-13, 0.0]
 [5.77589e-18, 0.0, 0.0]              [-4.33192e-18, 0.0, 0.0]

In [56]:
# assume r to coincide with midpoint of the element
N = 5; h = 1/N;
xmesh = Vector(0:h:1)
ymesh = Vector(0:h:1)
result = fill(zeros(3),N,N)
for (i,xi) in enumerate(xmesh[1:end-1]) 
    for (j,yj) in enumerate(ymesh[1:end-1])  
      r = Point3D(xi,yj,-t)
      result[i,j] = hcubature(rp -> vp_density(r,rp,M), (xi, yj, -t), (xi+h, yj+h, t))[1]
    end 
end